# **3-1-1 - Street Cleaning Requests in San Francisco**
A deep dive into the defilement of San Francisco

# Table of Contents
1. [Motivation](#Motivation)
2. [Basic Statistics](#Basic-Statistics)
    - [Dataset 1 - 311 Cases in San Francisco](#Dataset-1---311-Cases-in-San-Francisco)
    - [Dataset 2 - Police Department Incident Reports: Historical 2003 to May 2018](#Dataset-2---Police-Department-Incident-Reports:-Historical-2003-to-May-2018)
    - [Dataset 3 - Police Department Incident Reports: 2018 to Present](#Dataset-3---Police-Department-Incident-Reports:-2018-to-Present)
3. [Data analysis](#Data-Analysis)
4. [Genre](#genre)
5. [Visualizations](#visualizations)
6. [Discussion](#discussion)
7. [Contributions](#contributions)

This notebook serves as a behind-the-scenes explanation of our project. It is meant to be read as a supplement to our story on 3-1-1 Street Cleaning Requests in San Francisco that is posted [here](https://gustavhansen99.github.io/SocialDataFinalProject/) in the case that you felt more curious about the technical details of our project. However, it is not necessary to read this notebook in order to understand our website.

In order to run the notebook all libraries defined below must be installed and loaded on your machine.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Motivation

# Basic statistics

## Dataset 1 - 311 Cases in San Francisco

## Dataset 2 - Police Department Incident Reports: Historical 2003 to May 2018

## Dataset 3 - Police Department Incident Reports: 2018 to Present

# Data Analysis

# Genre

# Visualizations

# Discussion

# Contributions